In [9]:
import pandas as pd
import sqlite3
query = 'SELECT * FROM rose_wine'

conn = sqlite3.connect('wine_database.db')
cursor = conn.cursor()
df = pd.read_sql_query(query, conn)
df['Name'] = df['Name'].str.encode('iso-8859-1').str.decode('utf-8')
df['Region'] = df['Region'].str.encode('iso-8859-1').str.decode('utf-8')
df['Winery'] = df['Winery'].str.encode('iso-8859-1').str.decode('utf-8')
df

,Name,Country,Region,Winery,Rating,Price,Year
0,Mad Méditerranée Rosé 2019,France,Méditerranée,Château Camparnaud,4.0,10.11,2019
1,Redoma Rosé 2019,Portugal,Douro,Niepoort,3.9,12.81,2019
2,Fantini Cerasuolo d'Abruzzo 2019,Italy,Abruzzo,Farnese,3.6,5.60,2019
3,Mount Hermon Rosé 2017,Israel,Galilee,Hermon,3.4,12.14,2017
4,La Villa Rosé 2018,France,Languedoc,Château La Sauvageonne,4.6,68.95,2018
...,...,...,...,...,...,...,...
389,Rosé 2019,France,Coteaux d'Aix-en-Provence,AIX,4.0,13.90,2019
390,Rosa dei Frati Riviera del Garda Classico Rosé...,Italy,Lombardia,Cà dei Frati,4.1,11.20,2019
391,Côte des Roses Rosé 2019,France,Languedoc,Gérard Bertrand,3.9,9.36,2019
392,M Rosé 2019,France,Côtes de Provence,Minuty,4.0,12.85,2019


In [11]:
df.to_csv('scraped_Rose.csv', index=False)

In [12]:
dummies = pd.get_dummies(df["Name"])
dummies.head()
Counrty_dummies = pd.get_dummies(df["Country"])
Region_dummies = pd.get_dummies(df["Region"])
Winery_dummies = pd.get_dummies(df["Winery"])
empty_df = df.drop(columns=["Name", "Country", "Region", "Winery"])

In [13]:
df_scaled = pd.concat([empty_df, dummies, Region_dummies, Counrty_dummies, Winery_dummies], axis =1)
df_scaled.head()

,Rating,Price,Year,12 e Mezzo del Salento Rosato 2018,1701 Rosé 2018,281 Rosé 2019,3055 Rosé 2017,3055 Rosé 2018,Aire de Protos Rosé 2018,Aire de Protos Rosé 2019,...,Viña Albali,Viña San Juan,Viñas del Vero,Weingut Bründlmayer,Weingut Metzger,Welmoed,Woodbridge by Robert Mondavi,Wölffer Estate,Zenato,Zeni
0,4.0,10.11,2019,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3.9,12.81,2019,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3.6,5.60,2019,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3.4,12.14,2017,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4.6,68.95,2018,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

In [15]:
from sklearn.model_selection import train_test_split
X = df_scaled.drop(columns="Rating")
y = df_scaled["Rating"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [16]:
model.fit(X, y)

LinearRegression()

In [17]:
LinearRegression()

LinearRegression()

In [18]:
predicted_y_values = model.predict(X)

In [19]:
copy_df = df.copy()

In [47]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
score = model.score(X, y, sample_weight=None)
r2 = r2_score(y, predicted_y_values)
mse = mean_squared_error(y, predicted_y_values)
rmse = np.sqrt(mse)
std = np.std(y)

# Print releveant metrics.
print(f"The score is {score}.")
print(f"The r2 is {r2}.")
print(f"The mean squared error is {mse}.")
print(f"The root mean squared error is {rmse}.")
print(f"The standard deviation is {std}.")

The score is 0.9922718397435492.
The r2 is 0.9922718397435492.
The mean squared error is 0.0005762257592126624.
The root mean squared error is 0.024004702856162633.
The standard deviation is 0.2730601187318576.


In [21]:
Region_ratings = df.groupby("Region")["Rating"].mean()
Region_ratings

NameError: name 'df' is not defined

In [22]:
Year_ratings = df.groupby("Year")["Rating"].mean()

In [23]:
import matplotlib.pyplot as plt
plot_pandas = Year_ratings.plot.bar(color='lightpink')
plt.title("Rosé Wine Average Vintage Rating")
plt.xlabel('Vintage')
plt.ylabel('Average Rating')
plt.ylim(3, 5)
plt.show()

In [24]:
Rating_price = df.groupby("Rating")["Price"].mean()

In [25]:
import seaborn as sns
plt.figure(figsize=(13,5))

graph = sns.regplot(x=np.log(df['Price']), y='Rating', 
                    data=df, fit_reg=False, color='lightpink')
graph.set_title("Rosé Wine Rating by Price Distribuition", fontsize=20)
graph.set_xlabel("Price", fontsize= 15)
graph.set_ylabel("Rating", fontsize= 15)
graph.set_xticklabels(np.exp(graph.get_xticks()).astype(int))
plt.savefig('Rose_Price.png')
plt.show()

In [26]:
Country_ratings = df.groupby("Country")["Rating"].mean()

In [27]:
Country_price = df.groupby("Country")["Price"].mean()